In [ ]:
#| default_exp cli

# Command Line Tools

In [ ]:
#| export
from fastcore.utils import *
from fastcore.script import call_parse, bool_arg
from fasthtml.ft import *
from subprocess import check_output, run
import pathlib
import importlib
import httpx
import json

In [ ]:
#| export
@call_parse
def railway_link():
    "Link the current directory to the current project's Railway service"
    j = json.loads(check_output("railway status --json".split()))
    prj = j['id']
    idxpath = 'edges', 0, 'node', 'id'
    env = nested_idx(j, 'environments', *idxpath)
    svc = nested_idx(j, 'services', *idxpath)

    cmd = f"railway link -e {env} -p {prj} -s {svc}"
    res = check_output(cmd.split())

In [ ]:
#| export
def _run(a, **kw):
    print('#', ' '.join(a))
    run(a)

In [ ]:
#| export
@call_parse
def railway_deploy(
    name:str, # The project name to deploy
    mount:bool_arg=True # Create a mounted volume at /app/data?
):
    """Deploy a FastHTML app to Railway"""
    nm,ver = check_output("railway --version".split()).decode().split()
    assert nm=='railwayapp', f'Unexpected railway version string: {nm}'
    if ver2tuple(ver)<(3,8): return print("Please update your railway CLI version to 3.8 or higher")
    cp = run("railway status --json".split(), capture_output=True)
    if not cp.returncode: return print("This project is already deployed. Run `railway open`.")
    reqs = Path('requirements.txt')
    if not reqs.exists(): reqs.write_text('python-fasthtml')
    _run(f"railway init -n {name}".split())
    _run(f"railway up -c".split())
    _run(f"railway domain".split())
    railway_link.__wrapped__()
    if mount: _run(f"railway volume add -m /app/data".split())
    _run(f"railway up -c".split())

In [ ]:

def _airplane_tags(hdr: tuple, link_attr, asset_path):
    try:
        # print(hdr[2]['src'])
        script = httpx.get(hdr[2][link_attr])
    except httpx.HTTPError:
        script = ''
    fname = hdr[2][link_attr].split('/')[-1]
    fpath = Path(asset_path) / Path(fname)
    fpath.write_text(script)
    hdr[2][link_attr] = f"/{str(fpath)}"
    return hdr


In [ ]:
test_assets_dir = pathlib.Path('test-airplane-assets')
test_assets_dir.mkdir(exist_ok=True)
_airplane_tags(Link(rel='stylesheet', href='https://cdn.jsdelivr.net/npm/sakura.css/css/sakura.css', type='text/css'), 'href', 'tests-airplane-assets')
[x.unlink() for x in test_assets_dir.glob('*.*')]
test_assets_dir.rmdir()


In [ ]:
test_assets_dir.mkdir(exist_ok=True)
_airplane_tags(Script(src='/airplane-assets/htmx.min.js'), 'src', 'test-airplane-assets')
[x.unlink() for x in test_assets_dir.glob('*.*')]
test_assets_dir.rmdir()

In [ ]:
#| export
@call_parse
def airplane_mode(app: str = "main:app", asset_dir: str = 'airplane-assets'):
    module_name, app_name = app.split(':')
    module = importlib.import_module(module_name)
    app = getattr(module, app_name)
    pathlib.Path(asset_dir).mkdir(exist_ok=True)
    hdrs = []
    for hdr in app.router.hdrs:
        if hdr[0] == 'script': hdrs.append(_airplane_tags(hdr, 'src', asset_dir))
        if hdr[0] == 'link': hdrs.append(_airplane_tags(hdr, 'href', asset_dir))
    print(hdrs)

In [ ]:
hdrs = airplane_mode('.main:app')

TypeError: the 'package' argument is required to perform a relative import for '.main'

## Export -

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()

ModuleNotFoundError: No module named 'nbdev'